In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

import ribasim

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# # Define base_dir, results_dir and network name
# base_dir = Path("..\\..\\ribasim_lumping_data\\")
# results_dir = Path(base_dir, "results")
# network_name = "zutphen_tki_netwerk"

In [ ]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
dhydro_dir = Path(base_dir, "d-hydro")
results_dir = Path(base_dir, "results")

In [ ]:
network_name = "tki_zutphen_berkel"

simulation_name = 'tki_zuthpen_berkel_basis.dsproj_data' # nog verwerken in de code
file_bc = f'{base_dir}\\d-hydro\\{simulation_name}\\FlowFM\\input\\FlowFM_boundaryconditions1d.bc' # nog verwerken in de code

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden", f"{network_name}_afw_eenheden.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [ ]:
# # Load areas (discharge units: afwaterende eenheden)
# areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_selectie_Zutphen.shp")
# areas_gdf = gpd.read_file(areas_file_path)
# areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [ ]:
# Create networkanalysis
network = RibasimLumpingNetwork(
    name=network_name, 
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulation_output_dir="FlowFM\\output",
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

In [ ]:
simulation_name = 'tki_zuthpen_berkel_basis.dsproj_data'
file_bc = f'{base_dir}\\d-hydro\\{simulation_name}\\FlowFM\\input\\FlowFM_boundaryconditions1d.bc'

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data(file_bc)
# Export to geopackage
# network.export_to_geopackage()

In [ ]:
# Define locations where the network should be split into Ribasim basins:

network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151' # duikers voor wijk Zuidwijken
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
    ], 
    node_ids_to_include=[
        # 419, # voorbeeld splitsing
        # 1455, # extra punt rondom verdeelpunt De Berkel
    ],
    node_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

Export to ribasim

nodes

In [ ]:
# boundaries_gdf

In [ ]:
# set id's to node. Start with basins, then boundaries and then splitnodes. start with id 1
basins_gdf =network.basins_gdf.copy()
basins_gdf['node_id'] = basins_gdf['basin'] + 1
basins_gdf['type'] = 'Basin'

boundaries_gdf = network.boundaries_gdf.copy()
boundaries_gdf['node_id'] = boundaries_gdf['boundary_id'] + len(network.basins_gdf) +1
boundarynodetypes = {
    'dischargebnd': 'FlowBoundary', 
    'waterlevelbnd': 'LevelBoundary' ,
}
for nodetype in boundarynodetypes:
    boundaries_gdf.loc[boundaries_gdf['quantity']==nodetype, 'type'] = boundarynodetypes[nodetype]

splitnodes_gdf = network.split_nodes.copy()
splitnodes_gdf.insert(0, 'splitnode_id', range(len(splitnodes_gdf)))
splitnodes_gdf['node_id'] = splitnodes_gdf['splitnode_id'] + len(network.basins_gdf) + len(network.boundaries_gdf) +1
splitnodes_gdf['type'] = 'TabulatedRatingCurve' 
splitnodetypes = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'weir': 'TabulatedRatingCurve', 
    'culvert':'ManningResistance', 
    'manual': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve'
}
for nodetype in splitnodetypes:
    splitnodes_gdf.loc[splitnodes_gdf['split_type']==nodetype, 'type'] = splitnodetypes[nodetype]

# concat nodes
ribasim_node_gdf = pd.concat([basins_gdf, boundaries_gdf,splitnodes_gdf]).set_crs(basins_gdf.crs)
ribasim_node_gdf = ribasim_node_gdf.set_index('node_id')
ribasim_node_gdf = ribasim_node_gdf[['geometry', 'type']]
node = ribasim.Node(static=ribasim_node_gdf)

edges

In [ ]:
basin_connections_gdf = network.basin_connections_gdf[['mesh1d_node_id', 'basin_in','basin_out','geometry']]

# merge to find splitnode id
basin_connections_gdf = basin_connections_gdf.merge(splitnodes_gdf[['splitnode_id','mesh1d_node_id', 'node_id']], left_on='mesh1d_node_id', right_on='mesh1d_node_id')

# split connections in the connections upstream and downstream of splitnode
# add node ID's 
basin_connections_gdf_us = basin_connections_gdf.copy()
basin_connections_gdf_us['geometry'] = basin_connections_gdf_us.geometry.apply(lambda x: LineString([x.coords[0], x.coords[1]]))
basin_connections_gdf_us['from_node_id'] = basin_connections_gdf_us['basin_out'] +1
basin_connections_gdf_us['to_node_id'] = basin_connections_gdf_us['node_id']

basin_connections_gdf_ds = basin_connections_gdf.copy()
basin_connections_gdf_ds['geometry'] = basin_connections_gdf.geometry.apply(lambda x: LineString([x.coords[1], x.coords[2]]))
basin_connections_gdf_ds['from_node_id'] = basin_connections_gdf_ds['node_id']
basin_connections_gdf_ds['to_node_id'] = basin_connections_gdf_ds['basin_in'] +1

# boundary basin connections - add node ID's
boundary_basin_connections = network.boundary_basin_connections_gdf[['boundary_id', 'basin','geometry','boundary_location']].copy()

boundary_basin_connections_us = boundary_basin_connections.loc[boundary_basin_connections['boundary_location'] == 'upstream'].copy()
boundary_basin_connections_us['from_node_id'] = boundary_basin_connections_us['boundary_id']  + len(network.basins_gdf) +1
boundary_basin_connections_us['to_node_id'] = boundary_basin_connections_us['basin'] +1

boundary_basin_connections_ds = boundary_basin_connections.loc[boundary_basin_connections['boundary_location'] == 'downstream'].copy()
boundary_basin_connections_ds['from_node_id'] = boundary_basin_connections_ds['basin'] +1
boundary_basin_connections_ds['to_node_id'] = boundary_basin_connections_ds['boundary_id'] + len(network.basins_gdf) +1

# Setup the edges:
ribasim_edges = pd.concat([basin_connections_gdf_ds, basin_connections_gdf_us,boundary_basin_connections_us, boundary_basin_connections_ds]) 
ribasim_edges = ribasim_edges[['from_node_id','to_node_id','geometry']].reset_index()
ribasim_edges['from_node_id'].astype(int)

edge = ribasim.Edge(static=ribasim_edges)

basin

In [ ]:
dummyvalue = 5.5

# basin
profile_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin'].index.values.tolist()
    }
)
profile_data['storage'] = dummyvalue
profile_data['area'] = dummyvalue
profile_data['level'] = dummyvalue

static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin'].index.values.tolist()
    }
)
static_data['drainage'] = dummyvalue
static_data['potential_evaporation'] = dummyvalue
static_data['infiltration'] = dummyvalue
static_data['precipitation'] = dummyvalue
static_data['urban_runoff'] = dummyvalue

basin = ribasim.Basin(profile=profile_data, static=static_data)

rating curve

In [ ]:
# tabulated_rating_curve
static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='TabulatedRatingCurve'].index
    }
)
static_data['level'] = dummyvalue
static_data['discharge'] = dummyvalue

tabulated_rating_curve = ribasim.TabulatedRatingCurve(static=static_data)

In [ ]:
# manning_resistance
static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='ManningResistance'].index
    }
)
static_data['length'] = dummyvalue
static_data['manning_n'] = dummyvalue
static_data['profile_width'] = dummyvalue
static_data['profile_slope'] = dummyvalue

manning_resistance = ribasim.ManningResistance(static= static_data)

boundary

In [ ]:
# level_boundary
# static_boundary = boundaries_gdf[['node_id']].copy()

static_boundary = pd.DataFrame(
    data={
        "node_id": boundaries_gdf.loc[boundaries_gdf['quantity']=='waterlevelbnd']['node_id']
    }
)
static_boundary['level'] = dummyvalue

level_boundary = ribasim.LevelBoundary(static=static_boundary)

In [ ]:
# level_boundary
# static_boundary = boundaries_gdf[['node_id']].copy()

static_boundary = pd.DataFrame(
    data={
        "node_id": boundaries_gdf.loc[boundaries_gdf['quantity']=='dischargebnd']['node_id']
    }
)
static_boundary['flow_rate'] = dummyvalue

flow_boundary = ribasim.FlowBoundary(static=static_boundary)

pump

In [ ]:
# pump
static_pump = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Pump'].index.values.tolist()
    }
)
static_pump['flow_rate'] = 0.0

pump = ribasim.Pump(static=static_pump)

Export everything to geopackage

In [ ]:
# model
model = ribasim.Model(
    modelname="ribasim_model",
    node=node,
    edge=edge,
    basin=basin,
    level_boundary=level_boundary,
    pump=pump,
    tabulated_rating_curve=tabulated_rating_curve,
    manning_resistance=manning_resistance, 
    starttime="2020-01-01 00:00:00",
    endtime="2021-01-01 00:00:00",
)

# Write the model to a TOML and GeoPackage:
# model.write(f"{results_dir}/{network.name}")